# Read lagrangian trajectories and compute basic statistics

In [27]:
import numpy as np
import xarray as xr
import sys
sys.path.append('../')
from xdispersion import RelativeDispersion

drifters = xr.open_dataset('fort.1003.nc')

print(drifters)

<xarray.Dataset> Size: 6MB
Dimensions:   (particle: 1000, time: 201)
Coordinates:
  * particle  (particle) int64 8kB 0 1 2 3 4 5 6 ... 993 994 995 996 997 998 999
  * time      (time) int64 2kB 0 1 2 3 4 5 6 7 ... 194 195 196 197 198 199 200
Data variables:
    ID        (particle) int32 4kB ...
    x         (particle, time) float64 2MB ...
    y         (particle, time) float64 2MB ...
    ux        (particle, time) float64 2MB ...
    uy        (particle, time) float64 2MB ...


Now we initialize a `RelativeDispersion` class, with the `drifters` dataset and associated names for position and velocity. 

In [28]:
rd = RelativeDispersion(drifters, ragged=False, ID='particle',
                        xpos='x', uvel='ux', time='time',
                        ypos='y', vvel='uy', coord='cartesian')

print(rd)

pairs = rd.get_all_pairs()
pairs

 RelativeDispersion class with:
   xpos: x 
   ypos: y 
   uvel: ux 
   vvel: uy 
   time: time 
  coord: cartesian
  delta:  1.000
maxtlen: -1



ValueError: Dimensions {'ID'} do not exist. Expected one or more of FrozenMappingWarningOnValuesAccess({'particle': 1000, 'time': 201})